# Introduction to Cross Validation

In this lecture series we will do a much deeper dive into various methods of cross-validation. As well as a discussion on the general philosphy behind cross validation. A nice official documentation guide can be found here: https://scikit-learn.org/stable/modules/cross_validation.html

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data Example

In [2]:
df = pd.read_csv("../DATA/Advertising.csv")

In [3]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


----
----
----
## Train | Test Split Procedure 

0. Clean and adjust data as necessary for X and y
1. Split Data in Train/Test for both X and y
2. Fit/Train Scaler on Training X Data
3. Scale X Test Data
4. Create Model
5. Fit/Train Model on X Train Data
6. Evaluate Model on X Test Data (by creating predictions and comparing to Y_test)
7. Adjust Parameters as Necessary and repeat steps 5 and 6

In [4]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

**Create Model**

In [5]:
from sklearn.linear_model import Ridge

In [6]:
# Poor Alpha Choice on purpose!
model = Ridge(alpha=100)

In [7]:
model.fit(X_train,y_train)

Ridge(alpha=100)

In [8]:
y_pred = model.predict(X_test)

**Evaluation**

In [9]:
from sklearn.metrics import mean_squared_error

In [10]:
np.sqrt(mean_squared_error(y_test,y_pred))

2.7095711448556075

**Adjust Parameters and Re-evaluate**

In [11]:
model = Ridge(alpha=1)

In [12]:
model.fit(X_train,y_train)

Ridge(alpha=1)

In [13]:
y_pred = model.predict(X_test)

**Another Evaluation**

In [14]:
np.sqrt(mean_squared_error(y_test,y_pred))

1.5228334050147283

Much better! We could repeat this until satisfied with performance metrics. (We previously showed RidgeCV can do this for us, but the purpose of this lecture is to generalize the CV process for any model).

----
----
----
## Train | Validation | Test Split Procedure 

This is often also called a "hold-out" set, since you should not adjust parameters based on the final test set, but instead use it *only* for reporting final expected performance.

0. Clean and adjust data as necessary for X and y
1. Split Data in Train/Validation/Test for both X and y
2. Fit/Train Scaler on Training X Data
3. Scale X Eval Data
4. Create Model
5. Fit/Train Model on X Train Data
6. Evaluate Model on X Evaluation Data (by creating predictions and comparing to Y_eval)
7. Adjust Parameters as Necessary and repeat steps 5 and 6
8. Get final metrics on Test set (not allowed to go back and adjust after this!)

In [15]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

In [16]:
# SPLIT TWICE! Here we create TRAIN | VALIDATION | TEST

from sklearn.model_selection import train_test_split

# 70% of data is training data, set aside other 30%
X_train, X_OTHER, y_train, y_OTHER = train_test_split(X, y, test_size=0.3, random_state=101)

# Remaining 30% is split into evaluation and test sets
# Each is 15% of the original data size
X_eval, X_test, y_eval, y_test = train_test_split(X_OTHER, y_OTHER, test_size=0.5, random_state=101)

In [17]:
# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_eval = scaler.transform(X_eval)
X_test = scaler.transform(X_test)

**Create Model**

In [18]:
from sklearn.linear_model import Ridge

In [19]:
# Poor Alpha Choice on purpose!
model = Ridge(alpha=100)

In [20]:
model.fit(X_train,y_train)

Ridge(alpha=100)

In [21]:
y_eval_pred = model.predict(X_eval)

**Evaluation**

In [22]:
from sklearn.metrics import mean_squared_error

In [23]:
mean_squared_error(y_eval,y_eval_pred)

7.320101458823869

**Adjust Parameters and Re-evaluate**

In [24]:
model = Ridge(alpha=0.1)

In [25]:
model.fit(X_train,y_train)

Ridge(alpha=0.1)

In [26]:
y_eval_pred = model.predict(X_eval)

**Another Evaluation**

In [27]:
mean_squared_error(y_eval,y_eval_pred)

2.3591341032824715

**Final Evaluation (Can no longer edit parameters after this!)**

In [28]:
y_final_test_pred = model.predict(X_test)

In [29]:
mean_squared_error(y_test,y_final_test_pred)

2.242171297430173

----
----
----
## Cross Validation with cross_val_score

----

<img src="grid_search_cross_validation.png">

----

In [30]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [31]:
model = Ridge(alpha=100)

In [32]:
model

Ridge(alpha=100)

In [33]:
from sklearn.model_selection import cross_val_score

In [34]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv = 140)

In [35]:
scores

array([-2.70427692e-02, -3.31790835e+01, -5.65153004e-01, -1.04778022e+01,
       -7.34895640e+00, -1.10637655e+01, -8.38664903e-01, -1.60790934e-01,
       -8.65775142e-03, -4.69213304e+00, -1.12292006e+01, -7.96937798e-01,
       -2.74993212e-01, -4.35457647e-02, -1.24515615e+00, -2.04237018e+01,
       -8.31158014e-01, -3.37722779e+01, -3.35419609e+01, -1.21094434e+00,
       -4.47841924e-01, -1.50859420e-01, -1.31812883e+00, -2.27050231e+01,
       -1.10745538e+01, -1.30619920e-02, -7.95329945e+00, -2.00901573e+00,
       -1.12914336e+01, -7.48286305e-02, -2.02186114e+00, -3.94916922e+00,
       -5.38305964e+00, -1.02821772e+00, -1.67565233e+00, -7.70518996e-01,
       -4.47133589e-01, -1.62158222e-01, -8.76302679e-04, -4.66527651e-01,
       -4.99819651e-01, -2.71994502e+01, -2.85910080e-01, -7.49150072e+00,
       -2.17395638e-03, -4.87303305e+00, -7.99037404e+00, -1.82507608e+00,
       -1.18826881e+01, -2.14240184e+00, -3.63014520e-01, -9.26329918e-01,
       -1.20247294e-01, -

In [36]:
# Average of the MSE scores (we set back to positive)
abs(scores.mean())

7.095295396893961

In [37]:
def dispaly_scores(scores):
    print("Mean ", abs(scores.mean()))
    print("Std ", scores.std())
dispaly_scores(scores)

Mean  7.095295396893961
Std  12.937555201958663


**Adjust model based on metrics**

In [38]:
model = Ridge(alpha=1)

In [39]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=140)

In [40]:
# Average of the MSE scores (we set back to positive)
abs(scores.mean())

3.2924848066650134

In [41]:
l = []
for i in range(1,100):
    model = Ridge(alpha=i*0.01)
    scores = cross_val_score(model,X_train,y_train,scoring='neg_mean_squared_error',cv=140)
    l.append([(abs(scores.mean())),i*0.01])

In [42]:
min(l)

[3.2919644514993833, 0.33]

**Final Evaluation (Can no longer edit parameters after this!)**

In [43]:
# Need to fit the model first!
model.fit(X_train,y_train)

Ridge(alpha=0.99)

In [44]:
y_final_test_pred = model.predict(X_test)

In [45]:
np.sqrt(mean_squared_error(y_test,y_final_test_pred))

1.5227633737714947

----
----
----

# Cross Validation with cross_validate

The cross_validate function differs from cross_val_score in two ways:

It allows specifying multiple metrics for evaluation.

It returns a dict containing fit-times, score-times (and optionally training scores as well as fitted estimators) in addition to the test score.

For single metric evaluation, where the scoring parameter is a string, callable or None, the keys will be:
        
        - ['test_score', 'fit_time', 'score_time']

And for multiple metric evaluation, the return value is a dict with the following keys:

    ['test_<scorer1_name>', 'test_<scorer2_name>', 'test_<scorer...>', 'fit_time', 'score_time']

return_train_score is set to False by default to save computation time. To evaluate the scores on the training set as well you need to be set to True.

In [46]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [47]:
model = Ridge(alpha=100)

In [48]:
from sklearn.model_selection import cross_validate



In [49]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=140)

In [50]:
#scores

In [51]:
pd.DataFrame(scores).head()

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_max_error
0,0.000997,0.000998,-0.164447,-0.027043,-0.164447
1,0.000997,0.000000,-5.760129,-33.179084,-5.760129
2,0.000000,0.000997,-0.751767,-0.565153,-0.751767
3,0.000000,0.000998,-3.236943,-10.477802,-3.236943
4,0.000998,0.000000,-2.710896,-7.348956,-2.710896


In [52]:
pd.DataFrame(scores).mean()

fit_time                        0.000685
score_time                      0.000590
test_neg_mean_absolute_error   -1.999813
test_neg_mean_squared_error    -7.095295
test_max_error                 -1.999813
dtype: float64

**Adjust model based on metrics**

In [53]:
model = Ridge(alpha=1)

In [54]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=140)

In [55]:
pd.DataFrame(scores).mean()


fit_time                        0.000644
score_time                      0.000615
test_neg_mean_absolute_error   -1.300100
test_neg_mean_squared_error    -3.292485
test_max_error                 -1.300100
dtype: float64

In [56]:
model = Ridge(alpha=0.33)

In [57]:
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=140)

In [58]:
pd.DataFrame(scores).mean()

fit_time                        0.000703
score_time                      0.000560
test_neg_mean_absolute_error   -1.299814
test_neg_mean_squared_error    -3.291964
test_max_error                 -1.299814
dtype: float64

**Final Evaluation (Can no longer edit parameters after this!)**

In [59]:
# Need to fit the model first!
model.fit(X_train,y_train)

Ridge(alpha=0.33)

In [60]:
y_final_test_pred = model.predict(X_test)

In [61]:
mean_squared_error(y_test,y_final_test_pred)

2.3051860930107044

----
----